In [1]:
import ee
import geemap.core as geemap

from pph import opca

In [2]:
ee.Initialize()

In [3]:
scale = 30
region = ee.FeatureCollection("users/ryangilberthamilton/bq-trend-analysis/subset_BoQ_for_Ken_v2").geometry()

In [4]:
from geersd import Landsat8SR

image = Landsat8SR().filterBounds(region).filterDate("2019-04-01", "2019-10-31").filterClouds(1).select("SR_B.*").median()
band_names = image.select('SR_B.*').bandNames()

In [5]:
# Mean center the data to enable a faster covariance reducer
# and an SD stretch of the principal components.
mean_dict = image.reduceRegion(
    reducer=ee.Reducer.mean(), geometry=region, scale=scale, maxPixels=1e9
)
means = mean_dict.toImage(band_names)
centered = image.subtract(means)


In [6]:
# This helper function returns a list of new band names.
def get_new_band_names(prefix):
  seq = ee.List.sequence(1, band_names.length())

  def add_prefix_and_number(b):
    return ee.String(prefix).cat(ee.Number(b).int())

  return seq.map(add_prefix_and_number)

In [7]:
def get_principal_components(centered, scale, region):
  # Collapse bands into 1D array
  arrays = centered.toArray()

  # Compute the covariance of the bands within the region.
  covar = arrays.reduceRegion(
      reducer=ee.Reducer.centeredCovariance(),
      geometry=region,
      scale=scale,
      maxPixels=1e9,
  )

  # Get the 'array' covariance result and cast to an array.
  # This represents the band-to-band covariance within the region.
  covar_array = ee.Array(covar.get('array'))

  # Perform an eigen analysis and slice apart the values and vectors.
  eigens = covar_array.eigen()

  # This is a P-length vector of Eigenvalues.
  eigen_values = eigens.slice(1, 0, 1)
  # This is a PxP matrix with eigenvectors in rows.
  eigen_vectors = eigens.slice(1, 1)

  # Convert the array image to 2D arrays for matrix computations.
  array_image = arrays.toArray(1)

  # Left multiply the image array by the matrix of eigenvectors.
  principal_components = ee.Image(eigen_vectors).matrixMultiply(array_image)

  # Turn the square roots of the Eigenvalues into a P-band image.
  sd_image = (
      ee.Image(eigen_values.sqrt())
      .arrayProject([0])
      .arrayFlatten([get_new_band_names('sd')])
  )

  # Turn the PCs into a P-band image, normalized by SD.
  return (
      # Throw out an an unneeded dimension, [[]] -> [].
      principal_components.arrayProject([0])
      # Make the one band array image a multi-band image, [] -> image.
      .arrayFlatten([get_new_band_names('pc')])
      # Normalize the PCs by their SDs.
      .divide(sd_image)
  )


In [8]:
pca = get_principal_components(centered, scale, region)

In [9]:
pca.bandNames().getInfo()

['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7']

In [10]:
Map = geemap.Map()

Map.addLayer(pca.clip(region), {'min': -2, 'max': 2})

Map.centerObject(region, 10)

Map

Map(center=[44.074231310841725, -77.26133089882211], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [11]:
ee.batch.Export.image.toDrive(
    image=pca,
    region=region,
    folder='pca_test',
    fileNamePrefix="PCA-Export",
    scale=30,
    skipEmptyTiles=True,
).start()



In [12]:
from pph import opca

pca = opca.PrincipalCompoent().compute(region)

In [13]:
pca.bandNames()

In [14]:
Map = geemap.Map()

Map.addLayer(pca.clip(region), {'min': -2, 'max': 2})

Map.centerObject(region, 10)

Map

Map(center=[44.07423131084107, -77.26133089882178], controls=(ZoomControl(options=['position', 'zoom_in_text',…